# 方法設計

方法構成了Julia 程式當中的絕大多數的部份，也是 Julia 在多重分派中重要的角色。我們在設計方法的時候需要考慮的是廣義化，一個好的方法被設計出來，希望他不只能夠應用於少數情況，我們會希望他可以適用於更多的情境。我們希望一個方法可以被設計的夠俱備一般性，如此才能適用更多情境。

In [1]:
function plus(x, y)
    return x + y
end

plus (generic function with 1 method)

以上是一個乍看非常簡單，但是異常糟糕的方法實作。在 API 的部份，並沒有去界定 `x` 及 `y` 兩個參數的範圍，如此就允許接收各式各樣型別的物件，這樣很容易導致錯誤。如此一來，使用者也無從得知什麼時候該使用這個方法，便會以為任何型別的物件都可以進行運算，但是事實並非如此。

## API 設計 - 依目的設計

首先，我們需要界定參數型別的範圍。一般來說，需要依據設計者的設計目的而制定。例如：

In [2]:
function plus(x::Vector, y::Vector)
    return x + y
end

plus (generic function with 2 methods)

以上的實作多了一些資訊，但仍舊不足。`plus` 牽涉到的是 `Vector` 當中值的運算，但這邊並沒有去界定值的型別，我們可以利用參數化來處理，最廣義的形式是這個樣子。

In [3]:
function plus(x::Vector{T}, y::Vector{S}) where {T,S}
    return x + y
end

plus (generic function with 2 methods)

我們用參數化來處理，但是並沒有增加任何的限制，所以我們可以在後面加上型別的限制。

In [4]:
function plus(x::Vector{T}, y::Vector{S}) where {T<:Number,S<:Number}
    return x + y
end

plus (generic function with 3 methods)

如此我們完成了對 API 的設計，這是一個可以對一維陣列做加法運算的方法，陣列當中的值要是 `Number` 的子型別。

## 實作設計 - 依所支援的方法設計

接下來，我們需要檢查方法所接收的參數與內部實作之間的關係。參數有其型別，而我們也在 API 上定義的了型別，型別有各自的方法。我們需要確認我們所設計的 `plus` 中所使用的方法都是有支援的。

使用到的 API：

* `+`

在我們的範例中，`plus` 使用到的方法只有一個，就是 `+`。`+` 會直接作用在 `x` 及 `y` 身上，我們需要確認他們的型別 `Vector{T}` 及 `Vector{S}` 是有支援 `+` 的。這件事我們一般可以由官方文件查詢得到，或是我們可以直接執行並測試我們的實作得到驗證。

In [5]:
plus([1,2,3], [4,5,6])

3-element Array{Int64,1}:
 5
 7
 9

如果使用的是自己設計的方法或是套件，請再次確認被使用到的 API 是有實作的。

## 函式參數之間的依賴關係

考慮一個情境，我們在計算變異數的時候容許資料是中心化的（centralization，資料減去其平均值）或是尚未中心化。這樣可以方便使用者指定要中心化的平均值給函式計算。大概會寫出以下這樣的函式。

In [6]:
using Statistics: mean

function variance(a::Array, m=nothing)
    if !isnothing(m)
        m = mean(a, dims=1)
        b = a .- m
    else
        b = a
    end
    c = mean(b.^2, dims=1)
    c
end

variance (generic function with 2 methods)

一般會設置一個變數 `m` 來接收平均值，當沒有平均值時就將 `a` 指定給 `b`。這樣的作法會讓程式碼過於冗長。

我們利用 Julia 的語言特性讓計算平均值這件事自動執行，我們將 `m=mean(a, dims=1)` 直接作為參數的預設值使用，這邊值得注意的是，參數的計算依賴 `a`，所以參數 `a` 一定要在 `m` 之前。

In [7]:
function variance(a::Array, m=mean(a, dims=1))
    b = a .- m
    c = mean(b.^2, dims=1)
    c
end

variance (generic function with 2 methods)

作為預設值時，使用者可以不需要輸入要中心化的平均值，函式會自動計算平均並且中心化。

In [8]:
x = rand(5, 10)
variance(x)

1×10 Array{Float64,2}:
 0.0147769  0.0123791  0.122585  0.113098  …  0.0545233  0.0283248  0.0263375

使用者也可以手動給定平均值，那麼函式就會自動採用使用者輸入的平均值進行中心化。這麼做的同時保持程式碼的簡潔，也不會有效能損失。

In [9]:
variance(x, zeros(10)')

1×10 Array{Float64,2}:
 0.714695  0.588751  0.278634  0.51879  …  0.177243  0.572998  0.143986

可以在一些可提供預設值或是額外的計算資訊的情境下使用。不過這麼做會讓參數之間有依賴關係，參數之間的依賴關係是後者會依賴前者。我們這邊以一個簡單的例子來解釋這件事。

In [10]:
foo(a::Array, b=size(a), c=sum(b)) = c

foo (generic function with 3 methods)

In [11]:
x = rand(3, 4, 5);
foo(x)

12

如果調換了 `b` 及 `c` 的順序的話，會導致 `b` 尚未定義。

In [12]:
foo(a::Array, c=sum(b), b=size(a)) = c

foo (generic function with 3 methods)

In [13]:
foo(x)

UndefVarError: UndefVarError: b not defined

這樣的方式也可以用在解開（unpack）數組。如果指定了輸入的參數要是一個矩陣，那麼就會有兩個維度，這時候我們就可以將它的 `size` 解開成 `b` 及 `c`，已利後續利用。

In [14]:
foo(m::Matrix, (b, c)=size(m)) = b

foo (generic function with 5 methods)

In [15]:
y = rand(5, 6)
foo(y)

5